# キャリブレーション


In [1]:
PROGRAM_NAME = "07_calibration_v2"

In [2]:
from quri_parts_oqtopus.backend import OqtopusSseBackend
from oqtopus_sse_pulse.adapter import QuriAdapter, collect_payloads_from_job, collect_session_from_job
from oqtopus_sse_pulse.plotters import plot_payload

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

DEVICE = "anemone" # デバイス名 (勉強会ではanemoneに固定)
backend = OqtopusSseBackend() # OqtopusSseBackendのインスタンスを作成

def experiment(program: str) -> list:
    """
    指定した実験プログラムを実行し, 測定結果のペイロードを返す関数.
    """

    # ジョブの実行
    job = backend.run_sse(
        file_path=f"program/{program}.py",
        device_id=f"{DEVICE}-pulse",
        name=program,
    )
    print(f"job_id = {job.job_id}")

    # ジョブの完了待ち
    job.wait_for_completion()
    print(f"job_status = {job.status}  elapsed={job.execution_time}")

    # QuriAdapterの作成
    adapter = QuriAdapter(backend)

    # 測定結果の取得
    payloads = collect_payloads_from_job(
        backend=adapter,
        job_id=job.job_id,
    )
    
    return payloads

In [6]:
payloads = experiment(PROGRAM_NAME)

job_id = 06967671-323b-7eca-8000-3c1c0f0a5aab
job_status = succeeded  elapsed=958.011


In [7]:
from pprint import pprint

print(payloads[0]["status"])

pprint(payloads[0])

succeeded
{'calib_note': {'hpi_params': {'Q36': {'amplitude': 0.025897771532467712,
                                       'duration': 32.0,
                                       'target': 'Q36',
                                       'tau': 12,
                                       'timestamp': '2026-01-14 10:07:04'},
                               'Q37': {'amplitude': 0.036631062472105534,
                                       'duration': 32.0,
                                       'target': 'Q37',
                                       'tau': 12,
                                       'timestamp': '2026-01-14 10:07:12'},
                               'Q38': {'amplitude': 0.06914438765199243,
                                       'duration': 32.0,
                                       'target': 'Q38',
                                       'tau': 12,
                                       'timestamp': '2026-01-14 10:07:19'},
                               'Q39': {'amplitude': 

failedと表示されたらもう一度calibrationを行う。  
succeededなら次へ。

# calib_note, props, paramsの書き出し

以下のブロックでcalib_note, props, paramsの書き出しを行うので、必要な箇所を書き換えてpushする。

In [8]:
import json
import os
import yaml


if not os.path.exists("data"):
    os.makedirs("data")

with open(os.path.join("data", "calib_note.json"), "w") as f:
    json.dump(payloads[0].get("calib_note"), f, ensure_ascii=False, indent=2)
with open(os.path.join("data", "props.yaml"), "w", encoding="utf-8") as f:
    yaml.safe_dump(payloads[0].get("props"), f)
# with open(os.path.join("data", "params.yaml"), "w", encoding="utf-8") as f:
#     yaml.safe_dump(payloads[0].get("params"), f)
